# 使用colab云端运行自动字幕程序


第一步从github上将要用到的项目复制下来  
这里前面的!是为了告诉colab  
**这一步我们用的是shell命令而不是python语句**

In [1]:
!git clone https://github.com/fortypercnt/stream-translator.git

Cloning into 'stream-translator'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 54 (delta 29), reused 29 (delta 12), pack-reused 0
Unpacking objects: 100% (54/54), done.


下一步安装依赖

In [2]:
!pip install -r /content/stream-translator/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-6ltzjcoo
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-6ltzjcoo
     |████████████████████████████████| 5.3 MB 7.1 MB/s 
     |████████████████████████████████| 351 kB 62.7 MB/s 
     |████████████████████████████████| 163 kB 66.3 MB/s 
     |████████████████████████████████| 7.6 MB 43.2 MB/s 
     |████████████████████████████████| 41 kB 694 kB/s 
     |████████████████████████████████| 62 kB 1.6 MB/s 
     |████████████████████████████████| 2.3 MB 51.1 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 10.1 MB 49.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for whisper: filename=whisp

#修改工作

这里有一个小坑  
我们先尝试不做任何改动直接运行程序  
会发现在模型下载好之后直接报错找不到指定文件  
ValueError: The provided filename silero_vad.jit does not exist

In [3]:
!python3 stream-translator/translator.py https://www.youtube.com/watch?v=swe6ea_9P2s

Loading model...
100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 50.7MiB/s]
Traceback (most recent call last):
  File "stream-translator/translator.py", line 226, in <module>
    cli()
  File "stream-translator/translator.py", line 222, in cli
    main(url, **args)
  File "stream-translator/translator.py", line 117, in main
    vad = VAD()
  File "/content/stream-translator/vad.py", line 9, in __init__
    self.model = init_jit_model("silero_vad.jit")
  File "/content/stream-translator/vad.py", line 20, in init_jit_model
    model = torch.jit.load(model_path, map_location=device)
  File "/usr/local/lib/python3.7/dist-packages/torch/jit/_serialization.py", line 152, in load
    raise ValueError("The provided filename {} does not exist".format(f))  # type: ignore[str-bytes-safe]
ValueError: The provided filename silero_vad.jit does not exist


点击左侧的文件夹📂  
选择stream-translator  
右键silero_vad.jit获取文件地址  
双击vad.py可以看到右侧打开了一个在线编辑页面  
我们在第9行将init_jit_model后面括号里的内容  
改为刚才复制的文件地址  
然后ctrl+s保存后关闭文件  
这里是因为colab直接git下来的项目  
运行的时候这里会出现无法指定到正确路径的问题  
所以需要手动改为绝对路径

#开始运行

网址改为你想要实时转录的直播  
理论上来说只要streamlink支持的网站都可以

In [6]:
!python3 stream-translator/translator.py https://www.youtube.com/watch?v=arA8dig-S9o

Loading model...
Opening stream...
[cli][info] streamlink is running as root! Be careful!
[cli][info] Found matching plugin youtube for URL https://www.youtube.com/watch?v=arA8dig-S9o
[cli][info] Available streams: 144p (worst), 240p, 360p, 480p, 720p, 1080p (best)
[cli][info] Opening stream: 1080p (hls)
16:17:06 (ja)  but I am worried because there are too many time limit!
16:17:08 (ja)  Everybody got excited? It's very okay...
16:17:09 (ja)  I don't want to continue. I don't think everyone is interested.
16:17:10 (ja)  I want you to read it if you're interested.
16:17:11 (ja)  The story is really good.
16:17:19 (ja)  cosplay Sacrifice, Huh? Is it bad? Oppa's talents are in your mind. You gotta tell me.
16:17:23 (ja) 
Traceback (most recent call last):
  File "stream-translator/translator.py", line 226, in <module>
    cli()
  File "stream-translator/translator.py", line 222, in cli
    main(url, **args)
  File "stream-translator/translator.py", line 139, in main
    **decode_options)

忽略结尾因手动暂停而产生的报错信息  
转录出的结果看起来还是有些怪怪的  
我们一个个解决  


*   **转录出来的东西是英文**  
这是因为stream_translator的作者默认设置的是开启英文翻译  
运行的时候需要在后面加上参数 --task transcribe 
*   **有时候识别出来的语种并不是主播说的**  
语言参数默认为auto  
也就是每一句都是自动识别语种  
更改为指定语言可以解决这一问题
*   **不够精准**  
模型参数默认为small  
既然白嫖colab的GPU，我们可以修改为medium甚至large😀
*   **导入的流是1080P担心影响转录速度**  
正常来说ffmpeg会自动分离出音频流所以影响不大  
如果还是怕1080P的直播分片下载会卡  
这里可以手动去配置文件修改为worst拉取144p的流  
作者默认的audio_only参数对于youtube直播是无效的






在上一次曾经打开的文件夹里找到translator.py文件  
双击打开定位到167行def cli()这个函数里  
里面所有default对应的值就是默认值  
按照上面的需求我们可以按照如下修改  
--model medium  
--task transcribe  
--language Japanese（按照实际需求）  
--preferred_quality worst  
重新跑一遍试试看吧！  
注意加入新模型的时候要先下载

In [7]:
!python3 stream-translator/translator.py https://www.youtube.com/watch?v=arA8dig-S9o

Loading model...
100%|█████████████████████████████████████| 1.42G/1.42G [00:24<00:00, 62.8MiB/s]
tcmalloc: large alloc 1528012800 bytes == 0x6632000 @  0x7fde91cee1e7 0x4b2590 0x5ad01c 0x5dcfef 0x58f92b 0x590c33 0x5e48ac 0x4d20fa 0x51041f 0x58fd37 0x50c4fc 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x4bad0a 0x50e18c 0x58fd37 0x50c4fc 0x5b4ee6 0x6005a3 0x607796 0x60785c 0x60a436 0x64db82 0x64dd2e 0x7fde918ebc87 0x5b636a
Opening stream...
[cli][info] streamlink is running as root! Be careful!
[cli][info] Found matching plugin youtube for URL https://www.youtube.com/watch?v=arA8dig-S9o
[cli][info] Available streams: 144p (worst), 240p, 360p, 480p, 720p, 1080p (best)
[cli][info] Opening stream: 144p (hls)
16:34:19  やめきかないでな
16:34:23  オッシャとアロンね!オッシャから!
16:34:23  アラオカ
16:34:25  で、待ってやばいやばいその連携やばい!OKOK
16:34:27  ちょっとちょっとちょっとちょっとちょっと1個とった
16:34:33  見逃してえ、なんか、え?
16:34:38  こんなんこういう気がするのこいつにんにくばたけの
16:34:43  やばいやばいやばい死ぬ死ぬ死ぬ
16:34:49  いや待って犬の落とし物を信じていいのかわかんないナイスクリームでしょ
16:34:54  プレギシミ特に?逃げる逃げる
16:34:57 

😎你也来试试看吧！